In [1]:
import requests
import pandas as pd
import time
import random
import json
import re
import os
from google.colab import userdata
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

In [2]:
userdata = userdata.get('github')

os.environ['GITHUB_TOKEN'] = userdata

In [3]:
!git clone https://{userdata}@github.com/miguroi/sistech.git

Cloning into 'sistech'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 232.74 KiB | 927.00 KiB/s, done.


In [4]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [5]:
!ls

sample_data  sistech


In [7]:
!cd sistech

/content/sistech


In [46]:
!ls

coursera_courses.csv  README.md  sistech_scraping.ipynb


In [47]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   coursera_courses.csv

no changes added to commit (use "git add" and/or "git commit -a")


## Configuration & Setup

In [34]:
DELAY_RANGE = (1, 3)
DIFFICULTY_LEVELS = ['Beginner', 'Intermediate', 'Advanced', 'Mixed']
GRAPHQL_ENDPOINT = "https://www.coursera.org/graphql"
COURSES_PER_REQUEST = 12

SAMPLE_NUMBER = {
    'Beginner': 600,
    'Intermediate': 500,
    'Advanced': 400,
    'Mixed': 250
}

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Origin': 'https://www.coursera.org',
    'Referer': 'https://www.coursera.org/courses',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin'
})

## Utility Functions

In [35]:
def rate_limit():
    """Rate limiting with random delay"""
    delay = random.uniform(*DELAY_RANGE)
    time.sleep(delay)

## GraphQL Request Functions

In [36]:
def build_graphql_payload(difficulty_level, cursor="0", language=None):
    """Build GraphQL payload for course search with optional language filter"""
    # Build facet filters
    facet_filters = [[f"productDifficultyLevel:{difficulty_level}"]]

    # Add language filter if specified
    if language:
        facet_filters.insert(0, [f"language:{language}"])

    return [{
        "operationName": "Search",
        "variables": {
            "requests": [{
                "entityType": "PRODUCTS",
                "limit": COURSES_PER_REQUEST,
                "facets": ["topic", "skills", "productDifficultyLevel", "productDuration",
                          "productTypeDescription", "partners", "language", "subtitleLanguage", "isPartOfCourseraPlus"],
                "sortBy": "BEST_MATCH",
                "maxValuesPerFacet": 1000,
                "facetFilters": facet_filters,
                "cursor": cursor,
                "query": ""
            }]
        },
        "query": """
        query Search($requests: [Search_Request!]!) {
          SearchResult {
            search(requests: $requests) {
              ...SearchResult
              __typename
            }
            __typename
          }
        }

        fragment SearchResult on Search_Result {
          elements {
            ...SearchHit
            __typename
          }
          facets {
            ...SearchFacets
            __typename
          }
          pagination {
            cursor
            totalElements
            __typename
          }
          totalPages
          source {
            indexName
            recommender {
              context
              hash
              __typename
            }
            __typename
          }
          __typename
        }

        fragment SearchHit on Search_Hit {
          ...SearchArticleHit
          ...SearchProductHit
          ...SearchSuggestionHit
          __typename
        }

        fragment SearchArticleHit on Search_ArticleHit {
          aeName
          careerField
          category
          createdByName
          firstPublishedAt
          id
          internalContentEpic
          internalProductLine
          internalTargetKw
          introduction
          islocalized
          lastPublishedAt
          localizedCountryCd
          localizedLanguageCd
          name
          subcategory
          topics
          url
          skill: skills
          __typename
        }

        fragment SearchProductHit on Search_ProductHit {
          avgProductRating
          cobrandingEnabled
          completions
          duration
          id
          imageUrl
          isCourseFree
          isCreditEligible
          isNewContent
          isPartOfCourseraPlus
          name
          numProductRatings
          parentCourseName
          parentLessonName
          partnerLogos
          partners
          productCard {
            ...SearchProductCard
            __typename
          }
          productDifficultyLevel
          productDuration
          productType
          skills
          url
          videosInLesson
          translatedName
          translatedSkills
          translatedParentCourseName
          translatedParentLessonName
          tagline
          __typename
        }

        fragment SearchSuggestionHit on Search_SuggestionHit {
          id
          name
          score
          __typename
        }

        fragment SearchProductCard on ProductCard_ProductCard {
          id
          canonicalType
          marketingProductType
          badges
          productTypeAttributes {
            ... on ProductCard_Specialization {
              ...SearchProductCardSpecialization
              __typename
            }
            ... on ProductCard_Course {
              ...SearchProductCardCourse
              __typename
            }
            ... on ProductCard_Clip {
              ...SearchProductCardClip
              __typename
            }
            ... on ProductCard_Degree {
              ...SearchProductCardDegree
              __typename
            }
            __typename
          }
          __typename
        }

        fragment SearchProductCardSpecialization on ProductCard_Specialization {
          isPathwayContent
          __typename
        }

        fragment SearchProductCardCourse on ProductCard_Course {
          isPathwayContent
          rating
          reviewCount
          __typename
        }

        fragment SearchProductCardClip on ProductCard_Clip {
          canonical {
            id
            __typename
          }
          __typename
        }

        fragment SearchProductCardDegree on ProductCard_Degree {
          canonical {
            id
            __typename
          }
          __typename
        }

        fragment SearchFacets on Search_Facet {
          name
          nameDisplay
          valuesAndCounts {
            ...ValuesAndCounts
            __typename
          }
          __typename
        }

        fragment ValuesAndCounts on Search_FacetValueAndCount {
          count
          value
          valueDisplay
          __typename
        }
        """
    }]

def make_graphql_request(difficulty_level, cursor="0", language=None):
    """Make GraphQL request to Coursera API"""
    payload = build_graphql_payload(difficulty_level, cursor, language)

    try:
        rate_limit()
        response = session.post(GRAPHQL_ENDPOINT, json=payload, timeout=15)

        data = response.json()

        if not data or 'data' not in data[0]:
            print(f"    Invalid response structure")
            return {'courses': [], 'pagination': {'cursor': None, 'totalElements': 0}, 'total_pages': 0}

        search_result = data[0]['data']['SearchResult']['search'][0]

        return {
            'courses': search_result['elements'],
            'pagination': search_result['pagination'],
            'total_pages': search_result.get('totalPages', 1)
        }

    except Exception as e:
        print(f"    GraphQL request error: {e}")
        return {'courses': [], 'pagination': {'cursor': None, 'totalElements': 0}, 'total_pages': 0}

## Data Extraction Functions

In [37]:
def extract_course_from_graphql(course_data):
    """Extract course data from GraphQL response"""
    try:
        title = course_data.get('name', '')
        organization = ', '.join(course_data.get('partners', []))
        rating = course_data.get('avgProductRating')
        review_count = course_data.get('numProductRatings')
        skills = ', '.join(course_data.get('skills', []))
        duration = course_data.get('productDuration') or course_data.get('duration')
        difficulty = course_data.get('productDifficultyLevel', '')
        course_type = course_data.get('productType', '')
        url = course_data.get('url', '')
        is_free = course_data.get('isCourseFree', False)
        language = course_data.get('language', '')

        # Extract duration from tagline if not available
        if not duration:
            tagline = course_data.get('tagline', '')
            if tagline:
                duration_patterns = [
                    r'(\d+\s*-\s*\d+\s*(?:week|month|hour|day)s?)',
                    r'(\d+\s*(?:week|month|hour|day)s?)',
                ]

                for pattern in duration_patterns:
                    match = re.search(pattern, tagline, re.IGNORECASE)
                    if match:
                        duration = match.group(1)
                        break

        return {
            'title': title,
            'organization': organization,
            'rating': rating,
            'review_count': review_count,
            'difficulty': difficulty,
            'course_type': course_type,
            'duration': duration,
            'skills': skills,
            'url': url,
            'is_free': is_free,
            'language': language,
            'course_id': course_data.get('id', '')
        }

    except Exception as e:
        print(f"    Error extracting course data: {e}")
        return None

## Data Preprocessing Functions

In [38]:
def preprocess_courses(courses):
    """Clean and standardize course data"""
    if not courses:
        return courses

    df = pd.DataFrame(courses)

    if not df.empty:
        # Clean organization names
        df['organization'] = df['organization'].str.strip()

        # Standardize difficulty levels
        df['difficulty'] = df['difficulty'].str.title()

        # Clean course types
        df['course_type'] = df['course_type'].str.replace('_', ' ').str.title()

        # Clean skills
        df['skills'] = df['skills'].str.strip()
        df['skills'] = df['skills'].replace('', None)

        # Convert rating to float
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
        df['review_count'] = pd.to_numeric(df['review_count'], errors='coerce')

        return df.to_dict('records')

    return courses

## Web Scraping Functions

In [39]:
def scrape_difficulty_level_graphql(difficulty_level, max_courses=None, language=None):
    """Scrape courses for a difficulty level using GraphQL"""
    lang_suffix = f" ({language})" if language else ""
    print(f"Scraping {difficulty_level} courses{lang_suffix}...")

    if max_courses is None:
        max_courses = SAMPLE_NUMBER.get(difficulty_level, 500)

    all_courses = []
    cursor = "0"
    request_count = 0

    while len(all_courses) < max_courses:
        # Make GraphQL request
        result = make_graphql_request(difficulty_level, cursor, language)

        if not result['courses']:
            print(f"  No more courses found for {difficulty_level}{lang_suffix}")
            break

        # Extract course data
        batch_courses = []
        for course_data in result['courses']:
            course = extract_course_from_graphql(course_data)
            if course:
                batch_courses.append(course)

                if len(all_courses) + len(batch_courses) >= max_courses:
                    batch_courses = batch_courses[:max_courses - len(all_courses)]
                    break

        # Preprocess batch
        processed_courses = preprocess_courses(batch_courses)
        all_courses.extend(processed_courses)

        request_count += 1
        total_elements = result['pagination']['totalElements']

        print(f"  {difficulty_level}{lang_suffix} - Request {request_count}: {len(batch_courses)} courses | Total: {len(all_courses)}/{max_courses} | Available: {total_elements}")

        # Check if we should continue
        next_cursor = result['pagination']['cursor']
        if not next_cursor or next_cursor == cursor:
            print(f"  Reached end of results for {difficulty_level}{lang_suffix}")
            break

        cursor = next_cursor

    print(f"  {difficulty_level}{lang_suffix}: {len(all_courses)} courses collected")
    return all_courses

## Main Scraping Function

In [40]:
def scrape_coursera(levels=None, languages=None, max_requests_per_level=None, delay_range=(1, 3)):
    """
    Scrape Coursera courses using GraphQL API

    Args:
        levels: List of difficulty levels to scrape (default: all)
        languages: List of languages to filter by (default: None for all languages)
        max_requests_per_level: Max GraphQL requests per level (None = all available)
        delay_range: Tuple of (min, max) delay seconds (default: (1, 3))
    """
    global DELAY_RANGE
    DELAY_RANGE = delay_range

    if levels is None:
        levels = DIFFICULTY_LEVELS

    print("Starting Coursera scraping via GraphQL API...")
    if languages:
        print(f"Language filter: {', '.join(languages)}")
    start_time = time.time()

    all_courses = []

    # If languages are specified, scrape each language separately
    if languages:
        for language in languages:
            print(f"\n--- Scraping courses in {language} ---")
            for level in levels:
                level_courses = scrape_difficulty_level_graphql(level, max_requests_per_level, language)
                all_courses.extend(level_courses)
    else:
        # Scrape all languages together
        for level in levels:
            level_courses = scrape_difficulty_level_graphql(level, max_requests_per_level)
            all_courses.extend(level_courses)

    df = pd.DataFrame(all_courses)
    if df.empty:
        print("No courses found")
        return df

    # Remove duplicates
    original_count = len(df)
    df = df.drop_duplicates(subset=['course_id'], keep='first').reset_index(drop=True)

    elapsed = time.time() - start_time

    # Results summary
    print(f"\nResults:")
    print(f"  Total courses: {len(df)}")
    print(f"  Duplicates removed: {original_count - len(df)}")
    print(f"  Time: {elapsed/60:.1f} minutes")
    print(f"  With ratings: {df['rating'].notna().sum()}")
    print(f"  With skills: {df['skills'].notna().sum()}")
    print(f"  Organizations: {df['organization'].nunique()}")
    print(f"  Free courses: {df['is_free'].sum()}")

    # Difficulty breakdown
    print(f"\nDifficulty breakdown:")
    difficulty_counts = df['difficulty'].value_counts()
    for diff, count in difficulty_counts.items():
        print(f"  {diff}: {count}")

    # Language breakdown
    if 'language' in df.columns and df['language'].notna().any():
        print(f"\nLanguage breakdown:")
        language_counts = df['language'].value_counts()
        for lang, count in language_counts.head(10).items():
            print(f"  {lang}: {count}")

    # Skills sample
    skills_sample = df[df['skills'].notna()]['skills'].head(3).tolist()
    if skills_sample:
        print(f"\nSkills sample: {skills_sample[0][:60]}...")

    # Save to CSV
    filename = 'coursera_courses.csv'

    df.to_csv(filename, index=False)
    print(f"\nSaved: {filename}")

    return df

# Execution

In [41]:
if __name__ == "__main__":
    df = scrape_coursera(languages=['English'])

Starting Coursera scraping via GraphQL API...
Language filter: English

--- Scraping courses in English ---
Scraping Beginner courses (English)...
  Beginner (English) - Request 1: 12 courses | Total: 12/600 | Available: 6640
  Beginner (English) - Request 2: 12 courses | Total: 24/600 | Available: 6640
  Beginner (English) - Request 3: 12 courses | Total: 36/600 | Available: 6640
  Beginner (English) - Request 4: 12 courses | Total: 48/600 | Available: 6640
  Beginner (English) - Request 5: 12 courses | Total: 60/600 | Available: 6640
  Beginner (English) - Request 6: 12 courses | Total: 72/600 | Available: 6640
  Beginner (English) - Request 7: 12 courses | Total: 84/600 | Available: 6640
  Beginner (English) - Request 8: 12 courses | Total: 96/600 | Available: 6640
  Beginner (English) - Request 9: 12 courses | Total: 108/600 | Available: 6640
  Beginner (English) - Request 10: 12 courses | Total: 120/600 | Available: 6640
  Beginner (English) - Request 11: 12 courses | Total: 132/6

In [42]:
coursera_courses = pd.read_csv('coursera_courses.csv')
coursera_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1750 non-null   object 
 1   organization  1750 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1750 non-null   int64  
 4   difficulty    1750 non-null   object 
 5   course_type   1750 non-null   object 
 6   duration      1750 non-null   object 
 7   skills        1749 non-null   object 
 8   url           1750 non-null   object 
 9   is_free       1750 non-null   bool   
 10  language      0 non-null      float64
 11  course_id     1750 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 152.2+ KB


In [45]:
coursera_courses.head()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,url,is_free,language,course_id
0,Google Data Analytics,Google,4.763390,170291,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Data Storytelling, Rmarkdown, Data Literacy, D...",/professional-certificates/google-data-analytics,False,NaN,s12n~kr43OcbTEeqeNBKhfgCLyw
1,Google Cybersecurity,Google,4.822850,53304,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Threat Modeling, Network Security, Incident Re...",/professional-certificates/google-cybersecurity,False,NaN,s12n~Dy6K-2UKEe2PIRJn6nL9pQ
2,Google Project Management:,Google,4.842615,130704,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Quality Management, Project Management Life Cy...",/professional-certificates/google-project-mana...,False,NaN,s12n~fq9UWMbTEeqpthJ2RmWGow
3,Google AI Essentials,Google,4.835467,2749,Beginner,Specialization,ONE_TO_THREE_MONTHS,"Prompt Engineering, Generative AI, Artificial ...",/specializations/ai-essentials-google,False,NaN,s12n~3tzIujTqTk-YdcEFZ9r3sQ
4,Google Digital Marketing & E-commerce,Google,4.801080,40653,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Data Storytelling, Search Engine Marketing, Me...",/professional-certificates/google-digital-mark...,False,NaN,s12n~aYYrIEl-EeyCjQ5Y8Mzdsw
